# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset

In [2]:
splits = ['train','validation']
ds = {split: ds for split, ds in zip(splits, load_dataset('glue','sst2', split = splits))}
ds

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 67349
 }),
 'validation': Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 872
 })}

In [3]:
# Validate labels in the dataset and keep only valid ones
valid_ds = {}
for split in splits:
    valid_ds[split] = ds[split].filter(lambda example: example['label'] in [0, 1])
valid_ds

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/872 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 67349
 }),
 'validation': Dataset({
     features: ['sentence', 'label', 'idx'],
     num_rows: 872
 })}

In [4]:
#randomly select 10000 samples for the training set and 200 samples for the testing set
sample_ds = {}
sample_ds['train'] = ds['train'].shuffle(seed=42).select(range(10000))
sample_ds['validation'] = ds['validation'].shuffle(seed=42).select(range(200))


In [5]:
#load tokenizer and preprocess datasets
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
def preprocess_function(examples):
    return tokenizer(examples['sentence'],truncation = True, padding = True)

In [7]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = sample_ds[split].map(preprocess_function, batched = True)
print(tokenized_ds['train'][0])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

{'sentence': 'klein , charming in comedies like american pie and dead-on in election , ', 'label': 1, 'idx': 32326, 'input_ids': [74, 33663, 837, 23332, 287, 14577, 444, 588, 45630, 272, 2508, 290, 2636, 12, 261, 287, 3071, 837, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [8]:
print(tokenized_ds['train'][4])
print(tokenized_ds['validation'][4])

{'sentence': 'covered earlier and much better ', 'label': 0, 'idx': 35226, 'input_ids': [32111, 2961, 290, 881, 1365, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'sentence': "charles ' entertaining film chronicles seinfeld 's return to stand-up comedy after the wrap of his legendary sitcom , alongside wannabe comic adams ' attempts to get his shot at the big time . ", 'label': 1, 'idx': 426, 'input_ids': [10641, 829, 705, 17774, 2646, 16199, 2983, 384, 47187, 705, 82, 1441, 284, 1302, 12, 929, 

In [9]:
#load and setup the pretrained model
model = AutoModelForSequenceClassification.from_pretrained(
    'gpt2',
    num_labels = 2,
    id2label = {0:'negative',1:'positive'},
    label2id = {'negative':0,'positive':1}
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {'accuracy':(predictions == labels).mean()}

In [11]:
# Set the model's pad token id to match the tokenizer's pad token id
model.config.pad_token_id = tokenizer.pad_token_id
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [12]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [13]:
model.score

Linear(in_features=768, out_features=2, bias=False)

In [14]:
#the training arguments
training_args = TrainingArguments(
    output_dir = "./results",
    learning_rate = 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs = 1,
    weight_decay=0.01,
    evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
    save_strategy="epoch", # Model is saved at the end of each epoch.
    load_best_model_at_end=True, # Load the best model at the end of training.
)

pre_trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['validation'],
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics = compute_metrics,
)

In [15]:
pre_train_results = pre_trainer.evaluate()
print(pre_train_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 4.971271514892578, 'eval_accuracy': 0.465, 'eval_runtime': 1.6329, 'eval_samples_per_second': 122.482, 'eval_steps_per_second': 30.62}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [16]:
from peft import LoraConfig, get_peft_model, TaskType

In [17]:
model = AutoModelForSequenceClassification.from_pretrained('gpt2',
                                                      num_labels=2,
                                                      id2label={0: "NEGATIVE", 1: "POSITIVE"},
                                                      label2id={"NEGATIVE": 0, "POSITIVE": 1})

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
for param in model.parameters():
    param.requires_grad = True

In [19]:
model.config.pad_token_id = tokenizer.pad_token_id

In [20]:
#create a peft config and model
config = LoraConfig(
        r = 4,
        lora_alpha = 32,
        lora_dropout = 0.1,
        bias = 'none',
        task_type = 'SEQ_CLS', #sequence_classification
        target_modules=['c_attn', 'c_proj']) 

In [21]:
peft_model = get_peft_model(model, config)
peft_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 408,576 || all params: 124,848,384 || trainable%: 0.3272577400761551


In [22]:
# Rename 'label' to 'labels' 
tokenized_ds["train"] = tokenized_ds["train"].map(lambda x: {'labels': x['label']}, batched=True, remove_columns=['label'])
tokenized_ds["validation"] = tokenized_ds["validation"].map(lambda x: {'labels': x['label']}, batched=True, remove_columns=['label'])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [23]:
#use trainer for the training loop
training_args_lora = TrainingArguments(
    output_dir = "./lora_results",
    learning_rate = 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs = 1,
    weight_decay=0.01,
    evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
    save_strategy="epoch", # Model is saved at the end of each epoch.
    load_best_model_at_end=True, # Load the best model at the end of training.
)

trainer = Trainer(
    model = peft_model,
    args = training_args_lora,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['validation'],
    tokenizer = tokenizer,
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding=True, max_length = 512),
    compute_metrics = compute_metrics,
)

In [24]:
# Start the training process
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.500700,0.334081,0.885000


Checkpoint destination directory ./lora_results/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2500, training_loss=0.5711528198242187, metrics={'train_runtime': 191.8197, 'train_samples_per_second': 52.132, 'train_steps_per_second': 13.033, 'total_flos': 295010725736448.0, 'train_loss': 0.5711528198242187, 'epoch': 1.0})

In [25]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'eval_loss': 0.3340812623500824,
 'eval_accuracy': 0.885,
 'eval_runtime': 1.4024,
 'eval_samples_per_second': 142.613,
 'eval_steps_per_second': 35.653,
 'epoch': 1.0}

In [26]:
peft_model.save_pretrained('gpt-lora')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [27]:
import torch
from peft import AutoPeftModelForSequenceClassification

lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora", num_labels=2, ignore_mismatched_sizes=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Set the model's pad token id to match the tokenizer's pad token id
lora_model.config.pad_token_id = tokenizer.pad_token_id

In [29]:
training_args_lora = TrainingArguments(
    output_dir = "./inference",
    learning_rate = 2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs = 1,
    weight_decay=0.01,
    evaluation_strategy="epoch", # Evaluation is performed at the end of each epoch.
    save_strategy="epoch", # Model is saved at the end of each epoch.
    load_best_model_at_end=True, # Load the best model at the end of training.
)


In [30]:
finetuned_trainer = Trainer(
    model=lora_model,  # The fine-tuned PEFT model.
    args=training_args,# Training arguments, defined above.
    train_dataset=tokenized_ds["train"], # The tokenized training dataset.
    eval_dataset=tokenized_ds["validation"], # The tokenized evaluation dataset.
    tokenizer=tokenizer, # The tokenizer used for encoding the data.
    # Data collator that will dynamically pad the batches during training.
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics, # Function to compute metrics during evaluation.
)

In [31]:
# Evaluate the fine-tuned model on the validation set
finetuned_results = finetuned_trainer.evaluate()

# Print the evaluation results for the fine-tuned model
print("Evaluation results for the fine-tuned model:", finetuned_results)

Evaluation results for the fine-tuned model: {'eval_loss': 0.3340812623500824, 'eval_accuracy': 0.885, 'eval_runtime': 1.4577, 'eval_samples_per_second': 137.203, 'eval_steps_per_second': 34.301}


In [32]:
#the eval accuracy increased from 0.56 to 
#the eval loss decreased from 0.685